# Compile ONNX models

In [1]:
import onnx
import numpy as np
import tvm
import tvm.relay as relay
from tvm.contrib.download import download_testdata

## Load pretrained ONNX model

In [2]:
import tarfile

model_url = 'https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.onnx'
model_path = download_testdata(model_url, 'resnet50v2.onnx', module='onnx')

onnx_model = onnx.load(model_path)

File /Users/akitanak/.tvm_test_data/onnx/resnet50v2.onnx exists, skip.


## Load Labels

In [24]:
label_url = 'https://s3.amazonaws.com/onnx-model-zoo/synset.txt'
label_path = download_testdata(label_url, 'synset.txt', module='data')
with open(label_path) as f:
    labels = [l.rstrip() for l in f]


File /Users/akitanak/.tvm_test_data/data/synset.txt exists, skip.


## Load a test image

In [3]:
from PIL import Image
img_url = 'https://github.com/dmlc/mxnet.js/blob/master/data/cat.png?raw=true'
img_path = download_testdata(img_url, 'cat.png', module='data')
img = Image.open(img_path).resize((224, 224))

def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):  
         # for each pixel in each channel, divide the value by 255 to get value between [0, 1] and then normalize
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

img_arr = np.asarray(img)
img_arr = np.array([
    img_arr[:,:,0],
    img_arr[:,:,1],
    img_arr[:,:,2]
])
x = np.array([preprocess(img_arr)])


File /Users/akitanak/.tvm_test_data/data/cat.png exists, skip.


## Compile the model with relay

In [4]:
target = 'llvm'

input_name = 'data'
shape_dict = {input_name: x.shape}
sym, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with relay.build_config(opt_level=1):
    intrp = relay.build_module.create_executor('graph', sym, tvm.cpu(0), target)

In [5]:
dtype = 'float32'
tvm_output = intrp.evaluate(sym)(tvm.nd.array(x.astype(dtype)), **params).asnumpy()

In [36]:
import torch
output_tensor = torch.from_numpy(tvm_output)
labels[torch.argmax(output_tensor)]

'n02123159 tiger cat'